<a href="https://colab.research.google.com/github/fnavab/APS360-project/blob/main/CNN_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Network


Visualize some of the data by running the code below. Include the visualization in your writeup

In [ ]:
#mount googledrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import numpy as np
import time
import matplotlib.pyplot as plt

use_cuda=True
def get_data():
  from google.colab import drive
  drive.mount('/content/drive')
  #we might need to modify the transform
  transform1 = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
  data_set=torchvision.datasets.ImageFolder('/content/drive/My Drive/aps360/Lab_3b_Gesture_Dataset', transform=transform1)

  classes=data_set.classes
  return classes, data_set

Data Loading and Splitting

In [ ]:
# Prepare Dataloader
def get_data_loader( classes , batchsize, dataset):
  #data_loader = torch.utils.data.DataLoader(trainset, batch_size=batchsize, 
   #                                        num_workers=1, shuffle=True)

  # indices=[] #create a list to store all indices
  # for k in range(0,len(trainset)):
  #   indices.append(k)
 
  # Get the list of indices
  indices = np.arange(len(dataset))



# Split into train and validation  test
  np.random.seed(20) # Fixed numpy random seed for reproducible shuffling
  np.random.shuffle(indices)
  split = int(len(indices) * 0.6) #split at 60%
  split2=int(len(indices)*0.8)

#  # split into training and validationand test indices
  relevant_train_indices, relevant_val_indices  ,relevant_test_indices = indices[:split], indices[split:split2] , indices[split2:]
  train_sampler = SubsetRandomSampler(relevant_train_indices)
  train_loader = torch.utils.data.DataLoader(dataset, batch_size=batchsize)
                                      
  val_sampler = SubsetRandomSampler(relevant_val_indices)
  val_loader = torch.utils.data.DataLoader(dataset, batch_size=batchsize,
                                              num_workers=1, sampler=val_sampler)
  # # Get the list of indices to sample from
  
  test_sampler = SubsetRandomSampler(relevant_test_indices)
  test_loader = torch.utils.data.DataLoader(dataset, batch_size=batchsize,
                                          num_workers=1, sampler=test_sampler)
  
  return train_loader, val_loader, test_loader

In [ ]:
# # Verification Step - obtain one batch of images
def verify(data_loader,classes):
  dataiter = iter(data_loader)
  images, labels = dataiter.next()
  images = images.numpy()
  # plot the images in the batch, along with the corresponding labels
  fig = plt.figure(figsize=(25, 4))
  for idx in np.arange(20):
      ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
      plt.imshow(np.transpose(images[idx], (1, 2, 0)))
      ax.set_title(classes[labels[idx]])

In [ ]:
def save_features(data_loader,loader_name,feature_path, classes):
  n = 0
  for img, label in data_loader:
    features = alexnet.features(img)
    features_tensor = torch.from_numpy(features.detach().numpy())
    folder_name = feature_path + '/' +loader_name+'/'+ str(classes[label])
    if not os.path.isdir(folder_name):
      os.mkdir(folder_name)
    torch.save(features_tensor.squeeze(0), folder_name + '/' + str(n) + '.tensor')
    n += 1
  print(folder_name,"features saved")

In [ ]:
import torchvision.models
import os
alexnet = torchvision.models.alexnet(pretrained=True)

# initialize path

feature_path = '/content/drive/My Drive/aps360/Lab3/Lab3_Data/Features'
# Prepare Dataloader 
classes1, dataset= get_data()
train_loader, val_loader, test_loader=get_data_loader(classes=classes1, batchsize=1, dataset=dataset)

save_features(train_loader,"train",feature_path, classes=classes1)
save_features(val_loader,"val",feature_path, classes=classes1)
save_features(test_loader,"test",feature_path, classes=classes1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/aps360/Lab3/Lab3_Data/Features/train/I features saved
/content/drive/My Drive/aps360/Lab3/Lab3_Data/Features/val/F features saved
/content/drive/My Drive/aps360/Lab3/Lab3_Data/Features/test/D features saved


In [ ]:
def get_alex_data_loader(batch_size):
  train_dir = '/content/drive/My Drive/aps360/Lab3/Lab3_Data/Features/train'
  test_dir = '/content/drive/My Drive/aps360/Lab3/Lab3_Data/Features/test'
  val_dir = '/content/drive/My Drive/aps360/Lab3/Lab3_Data/Features/val'
  train_set = torchvision.datasets.DatasetFolder(train_dir, loader=torch.load, extensions=('.tensor'))
  val_set= torchvision.datasets.DatasetFolder(val_dir, loader=torch.load, extensions=('.tensor'))
  test_set = torchvision.datasets.DatasetFolder(test_dir, loader=torch.load, extensions=('.tensor'))
  torch.manual_seed(1) # set the random seed

  num_workers = 1
  train_feature_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, 
                                            num_workers=num_workers, shuffle=True)
  test_feature_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, 
                                            num_workers=num_workers, shuffle=True)
  val_feature_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, 
                                            num_workers=num_workers, shuffle=True)


  return train_feature_loader, test_feature_loader,val_feature_loader

In [ ]:
#modify batch size
train_feature_loader, test_feature_loader,val_feature_loader=get_alex_data_loader(64)

In [ ]:
# Verification Step - obtain one batch of features
dataiter = iter(val_feature_loader)
features, labels = dataiter.next()
print(features.shape)
print(labels.shape)

torch.Size([64, 256, 6, 6])
torch.Size([64])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Modify
class CNN_ALEX(nn.Module):
    def __init__(self):
        super(CNN_ALEX, self).__init__()
        self.name="CNN_ALEX"
        self.conv = nn.Conv2d(256, 10, 3)
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(10* 2* 2, 32)
        self.fc2 = nn.Linear(32, 9)

    def forward(self, x):
        x = self.pool(F.relu(self.conv(x)))
        x = x.view(x.shape[0], 10* 2* 2) #flatten feature data
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.squeeze(1)
        return x

In [ ]:
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

In [ ]:
def get_accuracy(model, data_loader):
    correct = 0
    total = 0
    for imgs, labels in data_loader:
        
        if use_cuda and torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()

        output = model(imgs)
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [ ]:
def train_net(model,target_classes,train_loader,val_loader, batch_size=64, learn_rate=0.001, num_epochs=30):
    ########################################################################
    ########################################################################
    # Fixed PyTorch random seed for reproducible result
    torch.manual_seed(1000)
    ########################################################################
    ########################################################################
    # Define the Loss function and optimizer
    # The loss function will be Cross Entropy,which will apply softmax to the output layer
    # Optimizer will be SGD with Momentum.
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learn_rate)

    ########################################################################
    # Set up some numpy arrays to store the training/test loss/erruracy
    train_acc = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_acc = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    ########################################################################
    # Train the network
    # Loop over the data iterator and sample a new batch of training data
    # Get the output from the network, and optimize our loss function.
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        total_train_loss = 0.0
        total_train_acc = 0.0
        total_epoch = 0
        for imgs, labels in iter(train_loader):

            if use_cuda and torch.cuda.is_available():
              imgs = imgs.cuda()
              labels = labels.cuda()

            # Forward pass, backward pass, and optimize
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()         # a clean up step for PyTorch

            # Calculate the statistics
            total_train_loss += loss.item()
            total_epoch += len(labels)
        train_loss[epoch] = float(total_train_loss) / (batch_size)
        train_acc[epoch]= get_accuracy(model, train_loader)
        val_acc[epoch] = get_accuracy(model, val_loader)
        print(("Epoch {}: Train acc: {}, Train loss: {} |"+
               "Validation acc: {}").format(
                   epoch + 1,
                   train_acc[epoch],
                   train_loss[epoch],
                   val_acc[epoch]
                   ))
        # Save the current model (checkpoint) to a file
        model_path = get_model_name(model.name, batch_size, learn_rate, epoch)
        torch.save(model.state_dict(), model_path)
    print('Finished Training')

    # Write the train/test loss/err into CSV file 
    epochs = np.arange(1, num_epochs + 1)
    np.savetxt("{}_train_acc.csv".format(model_path), train_acc)
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_acc.csv".format(model_path), val_acc)


    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))

    # plotting
    plt.title("Training Curve")
    plt.plot(epochs, train_loss, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(epochs, train_acc, label="Train")
    plt.plot(epochs, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()

Hyper parameter tuning

In [ ]:
use_cuda = True

model=CNN_ALEX()
#Modify
if use_cuda and torch.cuda.is_available():
    model.cuda()
train_net(model, classes1,train_feature_loader, val_feature_loader, batch_size=64, num_epochs=20, learn_rate = 0.001)

Epoch 1: Train acc: 0.6540518305224188, Train loss: 1.0324055682867765 |Validation acc: 0.6646090534979424
